In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite
import warnings
import sys
sys.path.append('C:\\Users\Reena\Documents\Reena\pyEvaluator-master')
from pyEvaluator.Evaluator import Evaluator
#from recsys.evaluation.decision import PrecisionRecallF1
#from recsys.evaluation.decision import PrecisionRecallF1
from datetime import datetime
from nxviz import CircosPlot

In [2]:
#Read all the csv files
donations = pd.read_csv('Donations.csv')
donors = pd.read_csv('Donors.csv',low_memory=False)
schools = pd.read_csv('Schools.csv',error_bad_lines=False)
teachers = pd.read_csv('Teachers.csv',error_bad_lines=False)
projects = pd.read_csv('Projects.csv',error_bad_lines=False)
resources = pd.read_csv('Resources.csv',error_bad_lines=False,warn_bad_lines=False)

In [3]:
donors.head()
print(donors.shape)
donors['Donor ID'].nunique()

(2122640, 5)


2122640

In [4]:
print(projects.shape)
projects['Project ID'].nunique()
projects.head()
projects['Project Subject Category Tree'].nunique()

(1110017, 18)


51

In [5]:
print(donations['Donor ID'].nunique())
donations.shape

2024554


(4687884, 7)

In [6]:
donations['Project ID'].nunique()

901965

In [7]:
#donations.loc[donations['Donor ID'].isin((donations.groupby('Donor ID').size().Value >= 1).index)]
df = donations[donations.groupby('Donor ID')['Donor ID'].transform('size') > 100]
df.shape

(722945, 7)

In [8]:
df_project = pd.merge(df, projects, on='Project ID', how='left')
df_project.shape

(722945, 24)

In [9]:
df_project.isnull().sum()

Project ID                                 0
Donation ID                                0
Donor ID                                   0
Donation Included Optional Donation        0
Donation Amount                            0
Donor Cart Sequence                        0
Donation Received Date                     0
School ID                              16994
Teacher ID                             16994
Teacher Project Posted Sequence        16994
Project Type                           16994
Project Title                          16996
Project Essay                          16994
Project Short Description              16994
Project Need Statement                 16995
Project Subject Category Tree          17024
Project Subject Subcategory Tree       17024
Project Grade Level Category           16994
Project Resource Category              17027
Project Cost                           16994
Project Posted Date                    16994
Project Expiration Date                16994
Project Cu

In [10]:
df_project = df_project.dropna(how='any')

In [11]:
pd.options.display.float_format = "{:.2f}".format
df_project.isnull().sum()

Project ID                             0
Donation ID                            0
Donor ID                               0
Donation Included Optional Donation    0
Donation Amount                        0
Donor Cart Sequence                    0
Donation Received Date                 0
School ID                              0
Teacher ID                             0
Teacher Project Posted Sequence        0
Project Type                           0
Project Title                          0
Project Essay                          0
Project Short Description              0
Project Need Statement                 0
Project Subject Category Tree          0
Project Subject Subcategory Tree       0
Project Grade Level Category           0
Project Resource Category              0
Project Cost                           0
Project Posted Date                    0
Project Expiration Date                0
Project Current Status                 0
Project Fully Funded Date              0
dtype: int64

In [12]:
pd.options.display.float_format = "{:.2f}".format
df_project.describe()

,Donation Amount,Donor Cart Sequence,Teacher Project Posted Sequence,Project Cost
count,663506.00,663506.00,663506.00,663506.00
mean,48.14,927.28,36.61,685.33
std,187.40,2127.57,54.99,2405.63
min,0.06,1.00,1.00,35.29
25%,1.00,78.00,3.00,305.59
50%,3.00,226.00,13.00,453.46
75%,25.00,767.00,46.00,670.82
max,60000.00,18115.00,497.00,255737.67


In [13]:
print(df_project.shape)
print(df_project['Project ID'].nunique())
print(df_project['Donor ID'].nunique())

(663506, 24)
192099
1822


In [14]:
df_final = df_project[['Donor ID', 'Project ID', 'Project Title']]

In [15]:
df_train, df_test = train_test_split(df_final, test_size=0.30, random_state=42)
print(df_train.shape)
print(df_test.shape)

(464454, 3)
(199052, 3)


In [16]:
warnings.filterwarnings('ignore')

# Get the duplicate records
#df_dups = df_train.duplicated(['Donor ID','Project ID'])
df_train = df_train.drop_duplicates(subset=['Donor ID', 'Project ID'])
#df_train = df_train.drop([df_train['Donor ID'] == 'c2d29ea8217363101da5e173cc06eefbd'])
#df_train.drop(df_train[df_train['is_duplicate'] == True].index, inplace=True)
#print((df_train["is_duplicate"] == True).sum())
df_train.head()

,Donor ID,Project ID,Project Title
171992,1756d766dab5b3cac8f2aa500430f326,3d6bf5ce473ed6980a86d676055b4d48,Professional Learning With Students in Mind!
476704,237db43817f34988f9d543ca518be4ee,a938cc8dd9942c946250734d64f84d0a,Move Your Bus!
613674,40c8b5781c5b591a8a0a646d8c513803,d8c59f8d7a62292ab25970c8d01630d1,Chromebooks for Differentiated Instruction
145917,237db43817f34988f9d543ca518be4ee,33dfd819f86b876adea7ebcb5b2c1f38,"Knock, Knock, Knocking on Donations for NoteKn..."
86481,99e91ab81ac57fc8fd3ccfd7da0a4889,1ee6ad53517b7950f4fd4bb282316ebb,A Carpet Spot for Everyone!


In [17]:
df_train['Donor ID'] = df_train['Donor ID'] + 'd'
df_test['Donor ID'] = df_test['Donor ID'] + 'd'
print(df_test.head())

                                 Donor ID                        Project ID  \
602992  b64afe5b7b5e07e1e8efdb7bc8913a5ed  d50c6873020eefe1258b693831756253   
519897  be833db4f16df25696356b1194a14308d  b877a4f95b0f050a3b4afbdfb29affaa   
385765  efdfbced227238f845028a1ff45cac90d  89ea58f1827e67601e68217e85f7ad53   
379654  16a9158ae440705ca92eee38148d65b1d  87b75689dbcf139c269d570aa2f3ed61   
242958  1be8ff2501f21dbe85a2fc87afb641e8d  5633bccf0cfc11bb65f79488133ced2a   

                                           Project Title  
602992                                Help Us Go Chrome!  
519897       Our Classroom Writing Center Needs Supplies  
385765                        Fitness is for Everyone!!!  
379654  Behind the Scenes, the Odds and Ends of Teaching  
242958         Listen Up!  Becoming Readers is Hard Work  


In [18]:
#Testing Data
df_train = df_train.sample(n=6000)
df_test = df_test.sample(n=2500)
#df_test = df_test.append({'Donor ID' : 'c2d29ea8217363101da5e173cc06eefbd' , 'Project ID' : '00d328a7d6eae881b1e004b6a42e8603'} , ignore_index=True)  

#print(df_train)
#print(df_test)
#df_test_eval = df_train.head(5)
#df_test1 = df_test.head(5)
#df_test1['Donor ID'] = df_test1['Donor ID'] + 'd'
#print(df_test_eval)
#df_test_eval
#df_test_eval = df_test_eval.append({'Donor ID' : '7c2ae0d7c1cc6b760cf1ac8473d52cf7d' , 'Project ID' : '23044192e3a8d66a85f8b256cea18d3c'} , ignore_index=True)  
#df_test1 = df_test1.append({'Donor ID' : '74f7db06bcd9557d1e992dc029db69ccd' , 'Project ID' : '17ac509d5f8668e5f4bbe5ebcc04438e'} , ignore_index=True)  


In [19]:
G = nx.Graph()
G.add_nodes_from(df_train['Donor ID'], bipartite=0)
G.add_nodes_from(df_train['Project ID'], bipartite=1)
G.add_edges_from([(row['Donor ID'], row['Project ID']) for idx, row in df_train.iterrows()])

#G.add_edges_from([('89938b0c7cca8e61dcd93d9363609bd1d', '3aacea50a59ee9a7a968addd2be334da')])

print(bipartite.is_bipartite(G))
G.number_of_selfloops()

True


0

In [20]:
# Get number of nodes in the Donor and Project partition:

def get_nodes_from_partition(G,partition):
    # Initialize an empty list for nodes to be returned
    nodes = []
    # Iterate over each node in the graph G
    for n in G.nodes():
        # Check that the node belongs to the particular partition
        if G.node[n]['bipartite'] == partition:
            # If so, append it to the list of nodes
            nodes.append(n)
    return nodes

# Print the number of nodes in the Donor partition
#print(len(get_nodes_from_partition(G,0)))

# Print the number of nodes in the Project partition
#print(len(get_nodes_from_partition(G,1)))


def shared_partition_nodes(G,node1, node2):
    # Check that the nodes belong to the same partition
    assert G.node[node1]['bipartite'] == G.node[node2]['bipartite']

    # Get neighbors of node 1: nbrs1
    nbrs1 = G.neighbors(node1)
    # Get neighbors of node 2: nbrs2
    nbrs2 = G.neighbors(node2)

    # Compute the overlap using set intersections
    overlap = set(nbrs1).intersection(nbrs2)
    return overlap

In [21]:
# Print the number of same projects between donors
#print(len(shared_partition_nodes(G,'74f7db06bcd9557d1e992dc029db69ccd','97b554d710370d87cc4b26c18263ca76d')))


def user_similarity(G, user1, user2, proj_nodes):
    # Check that the nodes belong to the 'users' partition
    assert G.node[user1]['bipartite'] == 0
    assert G.node[user2]['bipartite'] == 0

    # Get the set of nodes shared between the two users
    shared_nodes = shared_partition_nodes(G,user1,user2)

    # Return the fraction of nodes in the projects partition
    return len(shared_nodes) / len(proj_nodes)

# Compute the similarity score between users 'u4560' and 'u1880'
#project_nodes = get_nodes_from_partition(G,1)
#print(len(project_nodes))
#similarity_score = user_similarity( G,'74f7db06bcd9557d1e992dc029db69ccd' , '7c2ae0d7c1cc6b760cf1ac8473d52cf7d',project_nodes)

#print('similarity_score: ', '%.2f' % similarity_score)

In [22]:
from collections import defaultdict

def most_similar_users(G, user, user_nodes, proj_nodes):
    # Data checks
    assert G.node[user]['bipartite'] == 0

    # Get other nodes from user partition
    user_nodes = set(user_nodes)
    user_nodes.remove(user)

    # Create the dictionary: similarities
    similarities = defaultdict(list)
    for n in user_nodes:
        similarity = user_similarity(G, user, n, proj_nodes)
        similarities[similarity].append(n)
        #print(similarities)

    # Compute maximum similarity score: max_similarity
    #max_similarity = max(similarities.keys())
    #max(k for k, v in similarities.iteritems() if k != 0)
    max_similarity = max(similarities.keys())
    if max_similarity == 0:
        max_similarity = None

    # Return list of users that share maximal similarity
    return similarities[max_similarity]

#user_nodes = get_nodes_from_partition(G,0)
#project_nodes = get_nodes_from_partition(G, 1)

#print(most_similar_users(G,'31a0bef69915d4c5cbfeaab09fcad292d',user_nodes,project_nodes))

In [23]:
# Recommend

def recommend_repositories(G,from_user,to_user):
    # Get the set of repositories that from_user has contributed to
    from_repos = set(G.neighbors(from_user[0]))
    #print(from_repos)
    # Get the set of repositories that to_user has contributed to
    to_repos = set(G.neighbors(to_user))
    #print(to_repos)
    # Identify repositories that the from_user is connected to that the to_user is not connected to
    # Return a new graph that contains the edges that exist in G but not in H.

    return from_repos.difference(to_repos)

#Print the repositories to be recommended
#recommended = recommend_repositories(G ,'7c2ae0d7c1cc6b760cf1ac8473d52cf7d','74f7db06bcd9557d1e992dc029db69ccd')

In [24]:
def recommend_projects(G,donor_id, topn=10):
    

    user_nodes = get_nodes_from_partition(G,0)
    project_nodes = get_nodes_from_partition(G, 1)
    from_user = most_similar_users(G,donor_id,user_nodes,project_nodes)
    #print(from_user)
    if not from_user:
        return None
    recommended = recommend_repositories(G ,from_user,donor_id)
    #print(recommended);
    #myDict.update({donor_id:recommended})

    return recommended

#print(recommend_projects(G,'31a0bef69915d4c5cbfeaab09fcad292d', topn=10))

In [25]:
precision = 0
recall = 0
f1Score = 0
count = 0
for index, row in df_train.iterrows():
        actual = df_test['Project ID'][df_test['Donor ID'] == row['Donor ID']]
        recommendedProjects = recommend_projects(G,row['Donor ID'], topn=5)
        
        if (recommendedProjects == None or len(recommendedProjects) == 0):
            continue
        predicted = list(recommendedProjects)
        #print(row['Donor ID'])
        ev = Evaluator(gold_terms=set(actual), pred_terms=set(predicted))
        count = count +1
        precision = precision + ev.precision()
        recall = recall + ev.recall()
        f1Score = f1Score + ev.f1_score()
        
print("Precision: {}".format(precision/count))
print("Recall: {}".format(recall/count))
print("F1-Score: {}".format(f1Score/count))
print(count)

Precision: 0.0029962144602249806
Recall: 0.0029981503278964177
F1-Score: 0.0018891710585001293
3130
